# 04. 생성한 합성 데이터셋 검증

## Step 1. 통합 및 의미론적 중복 제거 (Semantic Deduplication)
### 목표: 생성된 파일에 흩어진 '같은 질문'을 찾아 하나만 남기고 삭제

### 방법:

1. 파일 병합 (Total Pool 생성).

2. 임베딩(Embedding) 기술로 질문 간의 유사도 측정(유사도가 0.85 이상인 질문들을 같은 그룹으로 묶고, 그중 답변 품질이 가장 좋은 1개만 남기고 삭제)

In [ ]:
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1. 데이터 로드 및 병합 (Data Loading)
file_paths = {
    0.1: 'qa_dataset_temp_0.1.json',
    0.2: 'qa_dataset_temp_0.2.json',
    0.3: 'qa_dataset_temp_0.3.json',
    0.4: 'qa_dataset_temp_0.4.json'
}

all_data = []
for temp, path in file_paths.items():
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        for entry in data:
            entry['temperature'] = temp # 출처 표시
        all_data.extend(data)

df = pd.DataFrame(all_data)

# 2. 임베딩 및 유사도 계산 (Embedding & Similarity)
# TF-IDF를 사용하여 텍스트를 벡터로 변환
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['question'])

# 코사인 유사도 행렬 계산 (질문 간 유사도)
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 3. 중복 제거 및 선별 (Deduplication & Selection)
visited = set()
deduplicated_data = []
threshold = 0.85 # 유사도 임계값 (0.85 이상이면 같은 질문으로 간주)

for i in range(len(df)):
    if i in visited:
        continue
    
    # 현재 질문과 유사한 질문들의 인덱스 찾기
    similar_indices = np.where(cosine_sim[i] > threshold)[0]
    
    # 유사한 질문들을 하나의 클러스터로 묶음
    cluster = df.iloc[similar_indices].to_dict('records')
    
    # 방문 처리 (중복 처리 방지)
    for idx in similar_indices:
        visited.add(idx)
    
    # [전략] 클러스터 내에서 'Temperature'가 가장 낮은(0.1) 데이터를 대표로 선정
    best_candidate = sorted(cluster, key=lambda x: x['temperature'])[0]
    deduplicated_data.append(best_candidate)

# 4. 결과 저장
output_df = pd.DataFrame(deduplicated_data)
output_df.to_json('deduplicated_dataset.jsonl', orient='records', lines=True, force_ascii=False)

print(f"제거 전: {len(df)}개 -> 제거 후: {len(output_df)}개")